In [3]:
import os
import json
import gc
from numba import jit
from tqdm import tqdm_notebook
from tqdm import tqdm

import lightgbm as lgb
import catboost as cbt

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
import time
import datetime
from datetime import datetime, timedelta

import gc
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
from collections import Counter
from statistics import mode

import warnings
warnings.filterwarnings("ignore")
import json
import math
from itertools import product
import ast
import re

# 科大讯飞移动广告反欺诈竞赛

本次比赛为参赛选手提供了5类数据：基本数据、媒体信息、时间、IP信息和设备信息。基本数据提供了广告请求会话sid，以及“是否作弊”的标识。媒体信息、时间、IP信息和设备信息等4类数据，提供了对作弊预估可能有帮助的辅助信息

In [6]:
############# 训练数据 ####################

path = '../datasets/'

## 数据合并，一起做特征工程
train1 = pd.read_table(path + "round1_iflyad_anticheat_traindata.txt")
train2 = pd.read_table(path + "round2_iflyad_anticheat_traindata.txt")

# reset_index重建索引，drop=True 是把原来索引删掉
train = train1.append(train2).reset_index(drop=True)
#train = pd.concat([train1,train2])

print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023389 entries, 0 to 3023388
Data columns (total 29 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sid           object 
 1   label         int64  
 2   pkgname       object 
 3   ver           object 
 4   adunitshowid  object 
 5   mediashowid   object 
 6   apptype       float64
 7   nginxtime     float64
 8   ip            object 
 9   city          object 
 10  province      float64
 11  reqrealip     object 
 12  adidmd5       object 
 13  imeimd5       object 
 14  idfamd5       object 
 15  openudidmd5   object 
 16  macmd5        object 
 17  dvctype       float64
 18  model         object 
 19  make          object 
 20  ntt           float64
 21  carrier       float64
 22  os            object 
 23  osv           object 
 24  orientation   float64
 25  lan           object 
 26  h             float64
 27  w             float64
 28  ppi           float64
dtypes: float64(10), int64(1), object(18)
memory u

In [8]:
############# 测试数据 #############################
test_2a = pd.read_table(path +
                        "round2_iflyad_anticheat_testdata_feature_A.txt")
test_2b = pd.read_table(path +
                        "round2_iflyad_anticheat_testdata_feature_B.txt")
# reset_index重建索引，drop=True 是把原来索引删掉
test = test_2a.append(test_2b).reset_index(drop=True)
#test = pd.concat([test_2a,test_2b])

test['label'] = -999  # 这样的方式是填充test 的label 列，使得test和train对其

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 29 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sid           object 
 1   pkgname       object 
 2   ver           object 
 3   adunitshowid  object 
 4   mediashowid   object 
 5   apptype       float64
 6   nginxtime     float64
 7   ip            object 
 8   city          object 
 9   province      float64
 10  reqrealip     object 
 11  adidmd5       object 
 12  imeimd5       object 
 13  idfamd5       object 
 14  openudidmd5   object 
 15  macmd5        object 
 16  dvctype       float64
 17  model         object 
 18  make          object 
 19  ntt           float64
 20  carrier       float64
 21  os            object 
 22  osv           object 
 23  orientation   float64
 24  lan           object 
 25  h             float64
 26  w             float64
 27  ppi           float64
 28  label         int64  
dtypes: float64(10), int64(1), object(18)
memory u

In [9]:
# reset_index重建索引，drop=True 是把原来索引删掉
data = train.append(test).reset_index(drop=True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5023389 entries, 0 to 5023388
Data columns (total 29 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sid           object 
 1   label         int64  
 2   pkgname       object 
 3   ver           object 
 4   adunitshowid  object 
 5   mediashowid   object 
 6   apptype       float64
 7   nginxtime     float64
 8   ip            object 
 9   city          object 
 10  province      float64
 11  reqrealip     object 
 12  adidmd5       object 
 13  imeimd5       object 
 14  idfamd5       object 
 15  openudidmd5   object 
 16  macmd5        object 
 17  dvctype       float64
 18  model         object 
 19  make          object 
 20  ntt           float64
 21  carrier       float64
 22  os            object 
 23  osv           object 
 24  orientation   float64
 25  lan           object 
 26  h             float64
 27  w             float64
 28  ppi           float64
dtypes: float64(10), int64(1), object(18)
memory u

## 特征工程

In [10]:
data.head(10)

,sid,label,pkgname,ver,adunitshowid,mediashowid,apptype,nginxtime,ip,city,...,make,ntt,carrier,os,osv,orientation,lan,h,w,ppi
0,d7460126-e071-4979-9ee8-42f72777a28a-156009070...,1,2d2664e827bcbb8b07100c7fbe072e9b,11.11.0,907d0f8c29663840491577a21c7b612a,ca64a500000d84c8fcb8a0587d0e1e0c,280.0,1.560091e+12,183.197.47.83,石家庄市,...,vivo,2.0,46000.0,Android,8.1.0,0.0,zh-CN,2340.0,1080.0,3.0
1,b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...,1,empty,NaN,10199dc8fea2e173525bc3151bd84312,3f2fc57a6e1f9c6fa4464c25cc1e88a3,319.0,1.560051e+12,106.34.14.149,开封市,...,HUAWEI,5.0,0.0,android,Android_9,0.0,NaN,1080.0,2040.0,0.0
2,f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...,0,16b81f93f4b1a35cebbf15f07683f171,3.2.1.0524.1958,83f2ecfe65f936f5f2ed59f8e8ff1d01,eea7280e1a2313e4c2e89290b01d196c,273.0,1.560089e+12,223.104.16.151,长春市,...,OPPO,2.0,46000.0,Android,8.1.0,1.0,zh-CN,2196.0,1080.0,0.0
3,fd60d096-f168-4540-b782-729d64d0fcc6-156006253...,0,empty,4.7.5,9f1eadd9092b19bc86ee0cacde1c867f,eec946a5a66c023ec9d3b2ede5900626,265.0,1.560063e+12,223.104.239.101,曲靖市,...,NaN,6.0,0.0,android,7.0,0.0,NaN,0.0,0.0,0.0
4,a037b032-a5c7-40ea-9161-26b118b12406-156007938...,1,cf4821986014a7fef9d7b7ad8de655e4,228,2af944462e43cd2f59acbbfd37445413,57b3053174973702549ba88b6017ac30,336.0,1.560079e+12,220.152.155.170,深圳市,...,EML-AL00,2.0,46000.0,Android,9,0.0,Zh-CN,2244.0,1080.0,0.0
5,7ac81a32-eefe-4fe9-9429-0ecb7be5d6b9-156006758...,1,170a88a12e36f8a0f1b73442304398b1,30928000,f67a95c5c748a5bc252d3e854f8e4977,d53d2af198ebef9544f0823c3c8e84f8,301.0,1.560068e+12,223.81.196.94,潍坊市,...,vivo,2.0,46000.0,Android,8.1.0,1.0,zh-CN,760.0,360.0,0.0
6,18deaade-64b3-4cc1-9fc8-e847f744d973-156004943...,0,b15030107319685361f4c8c0d9598cf1,NaN,4f2f5735b281a22587f22fb0aaba707e,651673a5531dde9cd40f08d09d16fe70,375.0,1.560049e+12,117.175.96.33,宜宾市,...,OPPO,2.0,46000.0,android,6.0.1,0.0,NaN,1280.0,720.0,0.0
7,a2d14df0-d499-4856-a330-8595f2923e21-156003109...,1,170a88a12e36f8a0f1b73442304398b1,30902000,4404268364e58fbf10c965a3bbc91924,d53d2af198ebef9544f0823c3c8e84f8,301.0,1.560031e+12,27.188.221.141,邯郸市,...,HUAWEI,2.0,46000.0,Android,9.0.0,1.0,zh-CN,747.0,360.0,0.0
8,c0f81859-8256-4474-bb32-2f3ed3700e19-156006715...,0,9fa14a798a23957226e066966594d7da,NaN,d0267547ad324d53078c09095c2e1148,06765782b6564446531bf6795f0fdc48,336.0,1.560067e+12,223.104.96.81,贵阳市,...,vivo,6.0,46000.0,android,8.1.0,0.0,NaN,0.0,0.0,0.0
9,db524c74-d9f7-41ed-bf2e-f38cdea4e15b-156005771...,1,3068122d33c1b72edb1f0d644f27405c,8.21.2,1ccb9164954c65e5e2fb5c7e176adb36,8832ab73ad09162a017fda9219ac2778,207.0,1.560058e+12,117.182.16.211,来宾市,...,OPPO,3.0,46000.0,android,8.1.0,1.0,zh-CN,1520.0,720.0,320.0


In [12]:
######## 1. 制造商预处理 ################
data['make']

#### 归类，同样的制造商可能有不同的值 ####

data.loc[data['make'].str.contains('长虹', na=False), 'make'] = 'changhong'
data.loc[data['make'].str.contains('朵唯', na=False), 'make'] = 'doov'
data.loc[data['make'].str.contains('sm', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('SAMSUNG', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('GT-', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('格力', na=False), 'make'] = 'gree'
data.loc[data['make'].str.contains('Moto G', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('Moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('摩托罗拉', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('诺基亚', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('Nokia', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('努比亚', na=False), 'make'] = 'nubia'
data.loc[data['make'].str.contains('美图', na=False), 'make'] = 'meitu'
data.loc[data['make'].str.contains('LG-', na=False), 'make'] = 'LG'
data.loc[data['make'].str.contains('联想', na=False), 'make'] = 'lenovo'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('小辣椒', na=False), 'make'] = 'xiaolajiao'
data.loc[data['make'].str.contains('HUAWEI', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('huawei', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('荣耀', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('华为', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('-L', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('al', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('Blade', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('BLADE', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('中兴', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('海信', na=False), 'make'] = 'hisense'
data.loc[data['make'].str.contains('Linux', na=False), 'make'] = 'Linux'
data.loc[data['make'].str.contains('乐丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('百立丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('乐视', na=False), 'make'] = 'letv'
data.loc[data['make'].str.contains('XT', na=False), 'make'] = 'Sony'
data.loc[data['make'].str.contains('htc', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('HTC', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('ASUS', na=False), 'make'] = 'Asus'
data.loc[data['make'].str.contains('锤子', na=False), 'make'] = 'smartisan'
data.loc[data['make'].str.contains('oppo', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('pb', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('realme', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('天语', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('Tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('酷派', na=False), 'make'] = 'coolpad'
data.loc[data['make'].str.contains('索尼', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('SONY', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('sony', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('小米', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('mi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2a', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m3', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m6', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('xiaomi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('redmi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('魅族', na=False), 'make'] = 'meizu'
data.loc[data['make'].str.contains('360', na=False), 'make'] = '360'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('赛博宇华', na=False), 'make'] = 'sop'
data.loc[data['make'].str.contains('金立', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('Gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('vivo', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('VIVO', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('oneplus', na=False), 'make'] = 'oneplus'
data.loc[data['make'].str.contains('一加', na=False), 'make'] = 'oneplus'

In [13]:
######## 2. 型号预处理 ################

# 将一些无处理的手机型号替代成能识别的手机信号
data['model'].replace('PACM00',"OPPO R15",inplace=True)
data['model'].replace('PBAM00',"OPPO A5",inplace=True)
data['model'].replace('PBEM00',"OPPO R17",inplace=True)
data['model'].replace('PADM00',"OPPO A3",inplace=True)
data['model'].replace('PBBM00',"OPPO A7",inplace=True)
data['model'].replace('PAAM00',"OPPO R15_1",inplace=True)
data['model'].replace('PACT00',"OPPO R15_2",inplace=True)
data['model'].replace('PABT00',"OPPO A5_1",inplace=True)
data['model'].replace('PBCM10',"OPPO R15x",inplace=True)

In [14]:
######## 3. 时间处理 #################

## nginxtime	bigint	请求到达服务时间，单位 ms
# 通过转化成时间戳转化为天和小时的特征
# 时间特征
data['time'] = pd.to_datetime(data['nginxtime'] / 1000,
                              unit='s') + timedelta(hours=8)
data['day'] = data['time'].dt.day
data['hour'] = data['time'].dt.hour

## 请求sid 里面有一个 到达的时间搓将时间搓切割出来
# sid 时间戳
data['sid_timestamp'] = data['sid'].apply(
    lambda x: str(x).split('-')[-1]).astype(str)

## 请求时间和到达时间相差
## 判断请求时间和到达时间是否相等
data['nginxtime_diff_sid_time'] = data['nginxtime'].astype(
    'float') - data['sid_timestamp'].astype('float')
data['nginxtime_sid_timestamp'] = (
    data['nginxtime'] == data['sid_timestamp'].astype(str)).astype(int)

In [15]:
######## 4. 将操作系统标准化 ####################

# 将 os 属性标准化，都转换成大写
data['os'] = data[data.os.notnull()]['os'].apply(lambda x: str(x).upper())
data['os'] = data['os'].map(lambda x: str(x).upper())

In [26]:
######## 5. 手机型号和手机制造商 ##################

data['big_model'] = data['model'].map(lambda x: str(x).split(' ')[0])

#制造商和型号是否相等
data['model_equal_make'] = (data['big_model'] == data['make']).astype(int)

# 制造商和机型进行交叉
data['machine'] = data['make'].astype(str) + '-' + data['model'].astype(str)

In [ ]:
######## 6. 手机尺寸信息处理 ##################

import re
data['h'].replace(0.0, np.nan, inplace=True)
data['w'].replace(0.0, np.nan, inplace=True)
# all_data['ppi'].replace(0.0, np.nan, inplace=True)
# cols = ['h', 'w', 'ppi']

cols = ['h', 'w']
gp_col = 'make'
for col in tqdm(cols):
    na_series = data[col].isna()
    names = list(data.loc[na_series, gp_col])
    
    # 使用均值 或者众数进行填充缺失值
    # df_fill = all_data.groupby(gp_col)[col].mean()
    df_fill = data.groupby(gp_col)[col].agg(lambda x: stats.mode(x)[0][0])
    
    t = df_fill.loc[names]
    
    t.index = data.loc[na_series, col].index
    # 相同的index进行赋值
    data.loc[na_series, col] = t

In [29]:
####### 7. ip 处理 ########################

data['ip0'] = data['ip'].apply(lambda x: '.'.join(str(x).split('.')[:1]))
data['ip1'] = data['ip'].apply(lambda x: '.'.join(str(x).split('.')[0:2]))
data['ip2'] = data['ip'].apply(lambda x: '.'.join(str(x).split('.')[0:3]))

data['reqrealip0'] = data['reqrealip'].apply(
    lambda x: '.'.join(str(x).split('.')[:1]))
data['reqrealip1'] = data['reqrealip'].apply(
    lambda x: '.'.join(str(x).split('.')[0:2]))
data['reqrealip2'] = data['reqrealip'].apply(
    lambda x: '.'.join(str(x).split('.')[0:3]))

In [30]:
# 判断二者是否相等
data['ip_reqrealip'] = (data['ip'] == data['reqrealip']).astype(int)
data['adidmd5_openudidmd5_d'] = (
    data['adidmd5'] == data['openudidmd5']).astype(int)

In [31]:
####### 8. 操作系统版本 ##############
data['osv0'] = data['osv'].astype(str).map(
    lambda x: '.'.join(x.split('.')[:1]))
data['osv1'] = data['osv'].astype(str).map(
    lambda x: '.'.join(x.split('.')[0:2]))
data['osv2'] = data['osv'].astype(str).map(
    lambda x: '.'.join(x.split('.')[0:3]))

In [32]:
####### 9. 特征进行交叉 ####################

#既然特征在catboost上面表现优越，在原来类别特征之间找新的交叉特征
### ip 暴力与设备进行暴力交叉
## 找城市，省份欺诈设备族群,ip 和 抬头ip过于稀疏，所以只拿province 和city 和设备
###['lan','make','model','osv','dvctype','imeimd5','os']暂时就这些，其他的nunique 过于少
#剔除重要性差特征
province_city = ['province', 'city']
pc_cos = ['make', 'model', 'osv', 'dvctype', 'imeimd5', 'lan', 'macmd5']  #os,
for i in province_city:
    for j in pc_cos:
        data['twoPC_' + str(i) + '_' +
             str(j)] = data[i].astype(str) + '_' + data[j].astype(str)
        
        
#媒体暴力交互
##媒体信息和设备交互欺诈族群寻找
###媒体信息有 ['pkgname','ver','adunitshowid','mediashowid'，'apptype']
###设备信息有['adidmd5','imeimd5','idfamd5','openudidmd5','macmd5','model','make','carrier','osv','lan']剔除了仅仅只有os
### 将媒体信息与全部设备信息进行暴力交叉
Media_Information = [
    'pkgname', 'ver', 'adunitshowid', 'mediashowid', 'apptype'
]  #按顺序开始取字段
Device_Information = [
    'adidmd5', 'imeimd5', 'openudidmd5', 'macmd5', 'model', 'make', 'carrier',
    'osv', 'lan'
]  #,'idfamd5'
for ii in Media_Information:
    for jj in Device_Information:
        data['twoMD_' + str(ii) + '_' +
             str(jj)] = data[ii].astype(str) + '_' + data[jj].astype(str)
        
        
        
#ip 和默认下载匹配app交互暴力交互ip
Ip_Information = ['ip', 'reqrealip']
Id_Information = ['adunitshowid']  #'mediashowid','mediashowid',好像过拟合
for ii in Ip_Information:
    for jj in Id_Information:
        data['twoMD_' + str(ii) + '_' +
             str(jj)] = data[ii].astype(str) + '_' + data[jj].astype(str)
        
        
        
## 强特'imeimd5'暴力交叉
##设备信息自身类别交互。先选出设备唯一id，一般设备id为International Mobile Equipment Identity IMEI识别码。
Za_Information = ['model', 'osv', 'ver', 'dvctype', 'carrier']  #按顺序开始取字段
Imd_Information = ['imeimd5']  #,'idfamd5',其他ID好像过拟合了
for ii in Za_Information:
    for jj in Imd_Information:
        data['twoMD_' + str(ii) + '_' +
             str(jj)] = data[ii].astype(str) + '_' + data[jj].astype(str)

        
##设备信息自身类别交互。通过操作系统和本身设备交互
### 看看ime5和其他设备和部分设备是如何交互
Device_Information3 = ['model', 'make', 'ntt']
for iiii in Device_Information3:
    data['twoMD_' + str('osv') + '_' +
         str(iiii)] = data['osv'].astype(str) + '_' + data[iiii].astype(str)

In [33]:
data['size'] = (np.sqrt(data['h']**2 + data['w']**2) / 2.54) / 1000
data['ratio'] = data['h'] / data['w']
data['px'] = data['ppi'] * data['size']
data['mj'] = data['h'] * data['w']

In [34]:
###查看该用户id用过多少个机型和设备等等(人与人的交互)开源特征：
sid_cro = [
    'ver', 'apptype', 'ntt', 'h', 'w', 'ppi', 'ratio', 'city', 'dvctype'
]
for x in sid_cro:
    data['sid' + '_' + 'use' + '_' + str(x) + '_count'] = data.groupby(
        [x])['sid'].transform('count')

In [35]:
############## 类别特征转换成数值特征 #######################

cat_col = [
    i for i in data.select_dtypes(object).columns if i not in ['sid', 'label']
]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [ ]:
############ 训练模型  ########################


feature_name = [i for i in data.columns if i not in ['sid', 'label', 'time', 'day']]
cat_list = [i for i in cat_col if i not in ['nginxtime']]

feature_importances = pd.DataFrame()
feature_importances['feature'] = feature_name
all_zero_feature=[]
X_train = data[data['label']!=-999]
y = data[data['label']!=-999]['label']
X_test = data[data['label']==-999]
print(X_train.shape,X_test.shape)

oof = np.zeros(X_train.shape[0])
prediction = np.zeros(X_test.shape[0])
seeds = [2018, 2019,4096, 2048, 1024]
num_model_seed = 1

for model_seed in range(num_model_seed):
    oof_cat = np.zeros(X_train.shape[0])
    prediction_cat=np.zeros(X_test.shape[0])
    skf = StratifiedKFold(n_splits=7, random_state=seeds[model_seed], shuffle=True)
    
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print(index)
        train_x, test_x, train_y, test_y = X_train[feature_name].iloc[train_index], X_train[feature_name].iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        print(train_x.shape)
        print(train_y.shape)
        print(test_x.shape)
        print(test_y.shape)
        cbt_model = cbt.CatBoostClassifier(iterations=5000,learning_rate=0.1,max_depth=7,
                                           l2_leaf_reg=1,verbose=100,early_stopping_rounds=500,task_type='GPU',eval_metric='F1',cat_features=cat_list)
        cbt_model.fit(train_x[feature_name], train_y,eval_set=(test_x[feature_name],test_y))            
        
        oof_cat[test_index] += cbt_model.predict_proba(test_x)[:,1]
        prediction_cat += cbt_model.predict_proba(X_test[feature_name])[:,1]/7       
        featureimportance=cbt_model.get_feature_importance(prettified=True)   
        importance_0=list(featureimportance[featureimportance['Importances']==0]['Feature Index'])
        all_zero_feature.extend(importance_0)
        
        feature_importances['fold-{}'.format(index+1)] = cbt_model.get_feature_importance()
        featureimportance.to_csv(path+'feats{}.csv'.format(index),index=False)  
        
    print('F1',f1_score(y, np.round(oof_cat)))    
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
print('score',f1_score(y, np.round(oof))) 
# write to csv
submit = test[['sid']]
submit['label'] =prediction
# submit.to_csv(path+"tmp/bs_gl1.csv",index=False)

submit['label'] = (prediction>=0.5).astype(int)
print(submit['label'].value_counts())

# submit.to_csv(path+"tmp/dt1.csv",index=False)

(3023389, 227) (2000000, 227)
0
(2591476, 223)
(2591476,)
(431913, 223)
(431913,)
0:	learn: 0.9369822	test: 0.9506234	best: 0.9506234 (0)	total: 1.25s	remaining: 1h 43m 47s
100:	learn: 0.9653390	test: 0.9673282	best: 0.9673282 (100)	total: 2m 15s	remaining: 1h 49m 50s
200:	learn: 0.9675168	test: 0.9692015	best: 0.9692015 (200)	total: 4m 32s	remaining: 1h 48m 33s
300:	learn: 0.9685575	test: 0.9697958	best: 0.9697958 (300)	total: 6m 53s	remaining: 1h 47m 34s
400:	learn: 0.9690024	test: 0.9701823	best: 0.9701869 (399)	total: 9m 10s	remaining: 1h 45m 14s
500:	learn: 0.9693755	test: 0.9703934	best: 0.9703934 (500)	total: 11m 24s	remaining: 1h 42m 29s
600:	learn: 0.9696560	test: 0.9704395	best: 0.9704705 (581)	total: 13m 41s	remaining: 1h 40m 13s
700:	learn: 0.9699012	test: 0.9705870	best: 0.9705907 (671)	total: 15m 56s	remaining: 1h 37m 46s
800:	learn: 0.9701570	test: 0.9707058	best: 0.9707058 (800)	total: 18m 14s	remaining: 1h 35m 38s
900:	learn: 0.9703205	test: 0.9707261	best: 0.9707565 (